<a href="https://colab.research.google.com/github/kartikchincholikar/Speech-Recognition-template-for-Anvil/blob/main/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading MOdel

In [ ]:
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer
!pip3 install --upgrade deepspeech
!pip3 install pysoundfile
!pip install tensorflow_io
!pip install pydub

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import soundfile
from pydub.utils import mediainfo
from deepspeech import Model
import wave 

In [ ]:


#mediainfo("checking.wav")
#mediainfo("checking_16.wav")
#mediainfo("8455-210777-0068.wav")

In [ ]:


ds = Model('/content/deepspeech-0.9.3-models.pbmm')
scorer_file_path = '/content/deepspeech-0.9.3-models.scorer'
ds.enableExternalScorer(scorer_file_path)
lm_alpha = 0.75
lm_beta = 1.85
ds.setScorerAlphaBeta(lm_alpha, lm_beta)
beam_width = 500
ds.setBeamWidth(beam_width)

0

In [ ]:
#text = ds.stt(load_audio("checking_16.wav"))

# ANVIL

In [ ]:
!pip install pydub
!pip install anvil-uplink

In [ ]:
import anvil
import anvil.server
import anvil.media
from anvil.media import TempFile
import anvil.js
from pydub import AudioSegment
import io

In [ ]:
anvil.server.connect("PASTE ANVIL UPLINK KEY HERE")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:


def load_audio(audio_path):
  fin = wave.open(audio_path, 'rb')
  audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)
  fin.close()
  return audio

#def frame_rate(audio_path):
#  fin = wave.open(audio_path, 'rb')
#  sample_rate = fin.getframerate()
#  fin.close()
#  return sample_rate

In [ ]:
@anvil.server.callable
def speech_to_text(media):

  
  print(f'url: {media.url}')
  print(f'content_type: {media.content_type}')
  print(f'length: {media.length} bytes')
  # This will be `None` since this is a website, not a file
  print(f'name: {media.name}')

  media_object = anvil.BlobMedia('audio/webm', media.get_bytes(), name='testing3.webm')
  with open('testing3.webm', 'wb+') as f:


    #Get bytes and set sampling rate
    song = AudioSegment.from_file(io.BytesIO(media_object.get_bytes()), format="webm")
    print(song.duration_seconds)
    song = song.set_frame_rate(16000)                                          ##########
    song.export("temp.wav", format="wav", bitrate="256k")

    # #PCM 32 to PCM 16
    data, samplerate = soundfile.read('temp.wav')
    soundfile.write('temp.wav', data, samplerate, subtype='PCM_16')  

    audio_file = "temp.wav"
    text = ds.stt(load_audio(audio_file))#, frame_rate(audio_file)      #########
    print(text)
    
  return text

In [ ]:
anvil.server.wait_forever()

url: None
content_type: audio/webm;codecs=opus
length: 21733 bytes
name: recording
3.78
i went to look at it
url: None
content_type: audio/webm;codecs=opus
length: 17931 bytes
name: recording
3.24
i have been to the garage
url: None
content_type: audio/webm;codecs=opus
length: 38038 bytes
name: recording
7.56
i have been to look at age
url: None
content_type: audio/webm;codecs=opus
length: 21102 bytes
name: recording
3.78
i went to the garage
url: None
content_type: audio/webm;codecs=opus
length: 13404 bytes
name: recording
2.34
i went to the garage
url: None
content_type: audio/webm;codecs=opus
length: 36748 bytes
name: recording
6.54
i would i i went to the garage


In [ ]:
#mediainfo("checking.wav")